In [2]:
from PIL import Image
import pandas as pd
import difflib
from bs4 import BeautifulSoup
from tqdm import tqdm
from IPython.display import FileLink, FileLinks

import pytesseract
import cv2
import math

# Set the path to the Tesseract OCR executable in the Colab environment
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [9]:
df_train_output_llm = pd.read_csv("/kaggle/input/additional-file-competition-locus/output_data_train.csv")
df_test_output_llm = pd.read_csv("/kaggle/input/additional-file-competition-locus/output_data_test.csv")
df_train = pd.read_csv("/kaggle/input/llm-competition-locus/train-20240120T074730Z-001/train/gt.csv")
df_test = pd.read_csv("/kaggle/input/llm-competition-locus/test-20240120T074730Z-001/test/gt.csv")

df_test_output_llm.fillna("", inplace=True)

In [11]:
df_train_output_llm.table[46]

'Qty Description Unit Price Amount\n10 Laser Mouse $950.00 $9,500.00\n20 Dual XL Monitors $150.00 $3,000.00\n02 Multi-jet Printer $150.00 $300.00\nSub Total $12,800.00\nDiscount $300.00\nTax $12.00\nTotal Value $1 2,51 2.00'

In [12]:
df_train_output_llm

,invoice_number,issue_date,original_issue_date,total,table,title
0,#982093,20/08/2022,20 August 2020,2112.25,DESCRIPTION QTY. PRICE TOTAL\nWeb Design 1 $20...,2dd5fc7e-4406-476c-9a23-838346d07d4f.jpg
1,NaN,0/07/2022,0 Jal 7022,NaN,Product Code] _Deseriton of Goods [HSCode [Uni...,56415ebf-98cc-4050-a4eb-58de6c5d22bf.jpg
2,us-001,11/10/2019,1110212019,154.06,ay DESCRIPTION UNIT PRICE AMOUNT 1 Front and r...,2b8dde51-be3e-4e5e-8495-cd730a0089cc.png
3,NaN,24/06/2020,24th June 2020,18.00,DESCRIPTION QTY. PRICE TOTAL\nEStore Beanie wi...,cf93684c-32cc-4cf4-9065-116ec75cc7e2.png
4,NaN,07/05/2005,2005/05/07,NaN,NaN,897aa322-67ca-4eb0-801c-6db864c6af49.jpg
5,us-001,11/02/2019,11/02/2019,154.06,aqy DESCRIPTION\n1 | Front and rear brake cabl...,c2206da4-2e43-43a7-89bd-0131da6c362e.png
6,00020,15/08/2017,15 Aug 2017,3696.00,ary Description vat Unit Price\nBooking Confir...,7c8910ed-2baa-43f2-9ecc-dfdeb7ddf159.png
7,IB1674,02/07/2002,2.07.02,312.50,Type Full Description Qty Units of Weight Unit...,31cc94f4-be3c-439b-953b-3a795aebe98e.png
8,RSN/18-19/4126,09/03/2019,9-Mar-2019,276900.00,Description of Goods HSNISAC GST Quantity Rate...,131f7202-9aae-4a9d-a928-e5e57f62b0ea.jpeg
9,A9GB28191212A017,06/12/2021,"6th December, 2021",5030.10,INumeer |v frricey |NT. |AMOUN Jevatue |rvee r...,d63180b8-4981-4956-8e7a-79eefb74a26d.jpg


In [13]:
df_test.columns

Index(['title', 'invoice_number', 'invoice_number_bbox', 'issue_date',
       'issue_date_bbox', 'total', 'total_bbox', 'table', 'table_bbox'],
      dtype='object')

In [14]:
def most_similar_word(input_word, word_list, similarity_ratio=0):
    # Get the most similar word in the list based on spelling
    similarity_ratio = similarity_ratio
    most_similar = None

    for word in word_list:
        current_ratio = difflib.SequenceMatcher(None, input_word, word).ratio()
        if current_ratio >= similarity_ratio:
            similarity_ratio = current_ratio
            most_similar = word

    return most_similar, similarity_ratio

In [15]:
def _extract_data_table(data, bounding_boxes, word_list):
    first_word = data.split()[0]
    last_word = data.split()[-1]    
    
    target_words = [first_word, last_word]
    bbox = []
    
    for target_word in target_words:
        flag = 0
        most_similar = most_similar_word(target_word, word_list, similarity_ratio=0)[0]
        if most_similar == None:
            bbox.append([])
        else:
            for item in bounding_boxes:
                if item['text'] == most_similar:
                    bbox.append(item['bounding_box'])
                    flag = 1
                    
                if flag:
                    break
                    
    
    flattened_list = [item for sublist in bbox for item in sublist]
    bbox = [flattened_list[0], flattened_list[1], flattened_list[-2], flattened_list[-1]]
    
    return bbox

In [16]:
def _extract_data_date(issue_date, bounding_boxes, word_list, issue_date_original):
    date_splits = issue_date.split()
    length = len(date_splits)

    bbox = []

    target_text = date_splits[0]
    ans = most_similar_word(target_text, word_list, similarity_ratio=0)[0]
#     print(ans)
    flag = 0
    for i, item in enumerate(bounding_boxes):
        if item['text'] == ans: 
            for j in range(length):
#                 print(bounding_boxes[i+j]['bounding_box'])
                bbox.append(bounding_boxes[i+j]['bounding_box'])

            flag = 1

        if flag:
            break

    flattened_list = [item for sublist in bbox for item in sublist]
    bbox = [flattened_list[0], flattened_list[1], flattened_list[-2], flattened_list[-1]]
    
    return bbox

In [17]:
def extract_data(data, bounding_boxes, word_list, field, issue_date_original=None):
    if field == "issue_date":
        bbox = _extract_data_date(data, bounding_boxes, word_list, issue_date_original)
    elif field == "table":
        bbox = _extract_data_table(data, bounding_boxes, word_list)
    else:
        most_similar = most_similar_word(data, word_list, similarity_ratio=0)[0]
        if most_similar == None:
            return "","[]"
        else:
            for item in bounding_boxes:
                if item['text'] == most_similar:
                    bbox = item['bounding_box']

    return data, str(bbox)

In [18]:
invoice_num_ans_all = []
invoice_nun_bbox_all = []
issue_date_ans_all = []
issue_date_bbox_all = []
total_ans_all = []
total_bbox_all = []
table_ans_all = []
table_bbox_all = []

In [21]:
len(df_train['title'].tolist())

61

In [22]:
invoice_num_ans_all = []
invoice_nun_bbox_all = []
issue_date_ans_all = []
issue_date_bbox_all = []
total_ans_all = []
total_bbox_all = []
table_ans_all = []
table_bbox_all = []

In [23]:
len(df_test['title'].tolist())

41

In [24]:
for i, title in enumerate(tqdm(df_test['title'].tolist())):
    img_path = "/kaggle/input/llm-competition-locus/test-20240120T074730Z-001/test/files/" + title
    
    img = Image.open(img_path)

    # Perform OCR and get hOCR output
    hocr_output = pytesseract.image_to_pdf_or_hocr(img, extension='hocr')

    # Save the hOCR output to a file
    with open('output_text.html', 'w', encoding='utf-8') as file:
        file.write(hocr_output.decode('utf-8'))
        
    with open('output_text.html', 'r', encoding='utf-8') as file:
        hocr_content = file.read()

    # Parse the hOCR content using BeautifulSoup
    soup = BeautifulSoup(hocr_content, 'html.parser')

    # Extract bounding box information from the parsed content
    bounding_boxes = []
    word_list = []
    for span in soup.find_all('span', class_='ocrx_word'):
        # Extract text
        text = span.get_text(strip=True)

        # Extract bounding box coordinates
        bbox = span['title'].split(';')[0].split(' ')[1:]
        bbox = [int(coord) for coord in bbox]

        bounding_boxes.append({
            'text': text,
            'bounding_box': bbox
        })

        word_list.append(text)
        
    img_index_output_llm = df_test_output_llm[df_test_output_llm['title'] == title].index[0]
    
    invoice_num = df_test_output_llm.iloc[img_index_output_llm]['invoice_number']
    issue_date = df_test_output_llm.iloc[img_index_output_llm]['issue_date']
    issue_date_original = df_test_output_llm.iloc[img_index_output_llm]['original_issue_date']
    total = df_test_output_llm.iloc[img_index_output_llm]['total']
    table = df_test_output_llm.iloc[img_index_output_llm]['table']
    
    if invoice_num == '':
        invoice_num_ans = ""
        invoice_num_bbox = "[]"
    else:
        invoice_num_ans, invoice_num_bbox = extract_data(invoice_num, bounding_boxes, word_list, "invoice_num")
    
    if issue_date == '':
        issue_date_ans = ""
        issue_date_bbox = "[]"
    else:
        issue_date_ans, issue_date_bbox = extract_data(issue_date, bounding_boxes, word_list, "issue_date", issue_date_original)
    
    if total == '':
        total_ans = ""
        total_bbox = "[]"
    else:
        total_ans, total_bbox = extract_data(str(total), bounding_boxes, word_list, "total")
        
    if table == '':
        table_ans = ""
        table_bbox = "[]"
    else:
        table_ans, table_bbox = extract_data(table, bounding_boxes, word_list, "table")
    
    invoice_num_ans_all.append(invoice_num_ans)
    invoice_nun_bbox_all.append(invoice_num_bbox)
    issue_date_ans_all.append(issue_date_ans)
    issue_date_bbox_all.append(issue_date_bbox)
    total_ans_all.append(str(total_ans))
    total_bbox_all.append(total_bbox)
    table_ans_all.append(table_ans)
    table_bbox_all.append(table_bbox)

100%|██████████| 41/41 [02:47<00:00,  4.08s/it]


In [25]:
df_test.columns

Index(['title', 'invoice_number', 'invoice_number_bbox', 'issue_date',
       'issue_date_bbox', 'total', 'total_bbox', 'table', 'table_bbox'],
      dtype='object')

In [26]:
df_test['invoice_number'] = invoice_num_ans_all
df_test['invoice_number_bbox'] = invoice_nun_bbox_all
df_test['issue_date'] = issue_date_ans_all
df_test['issue_date_bbox'] = issue_date_bbox_all
df_test['total'] = total_ans_all
df_test['total_bbox'] = total_bbox_all
df_test['table'] = table_ans_all
df_test['table_bbox'] = table_bbox_all

In [27]:
df_test.head()

,title,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,total,total_bbox,table,table_bbox
0,fd28f5e7-fc19-4f23-8dec-9a87b40f1a15.jpg,KBM/15-16/S0229,"[1119, 308, 1392, 349]",31/08/2015,"[1134, 343, 1286, 384]",310693.0,"[1274, 1325, 1388, 1358]",Product Description Amount TATA NANO TWIST XTA...,"[594, 593, 1388, 1358]"
1,c4141ea0-72cc-46f8-826f-a5d80451f8e5.jpg,Ne,"[283, 36, 294, 43]",19/11/2020,"[370, 48, 417, 57]",7000.0,"[444, 227, 483, 237]",Si Partculers Gusnity ale per Amount\n\n11) Re...,"[11, 190, 280, 43]"
2,77502028-18e6-4a51-9b7d-0cc5f4a68726.jpg,B202001676,"[620, 499, 720, 510]",07/03/2020,"[180, 891, 262, 912]",15576.0,"[713, 1967, 751, 1978]",S.No. Parts/OP code _—Part Labor Description H...,"[128, 746, 855, 840]"
3,e8d8c551-7a88-4f7d-bccf-4b205317b21e.jpg,123456789-SA,"[515, 1702, 741, 1729]",14/09/2017,"[1494, 1702, 1564, 1729]",918.0,"[2200, 3010, 2310, 3041]",Items in this shipment Ordered Shipment Follow...,"[200, 2063, 2310, 3041]"
4,a7e533f9-fea3-453e-b135-6b6d9d3c63a6.jpg,48,"[527, 566, 556, 586]",18/12/2014,"[529, 517, 671, 537]",121.0,"[1355, 1449, 1454, 1469]",Product Quantity Lda)\nSimple product 1 €100.0...,"[185, 910, 1454, 1469]"


In [28]:
df_test['total']

0      310693.0
1        7000.0
2       15576.0
3         918.0
4         121.0
5        349.85
6       4899.36
7         176.6
8     321113.82
9              
10       154.06
11        467.5
12       2550.0
13    2120000.0
14      23600.0
15      10440.0
16      2231.96
17        39.65
18       1010.0
19      62000.0
20      2112.25
21      80000.0
22      93000.0
23       6326.0
24       2860.0
25         70.0
26      1445.99
27      3618.95
28      34436.5
29         10.6
30        775.0
31        270.0
32       2407.0
33      50240.0
34         14.0
35       7200.0
36      4768.45
37       5475.0
38        635.9
39        120.0
40      13375.0
Name: total, dtype: object

In [29]:
import re

def remove_trailing_zeros(input_string):
    result = re.sub(r'\.?0*$', '', input_string)
    return result

In [30]:
df_test['total'] = df_test['total'].apply(lambda x: remove_trailing_zeros(x))

In [31]:
df_test

,title,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,total,total_bbox,table,table_bbox
0,fd28f5e7-fc19-4f23-8dec-9a87b40f1a15.jpg,KBM/15-16/S0229,"[1119, 308, 1392, 349]",31/08/2015,"[1134, 343, 1286, 384]",310693,"[1274, 1325, 1388, 1358]",Product Description Amount TATA NANO TWIST XTA...,"[594, 593, 1388, 1358]"
1,c4141ea0-72cc-46f8-826f-a5d80451f8e5.jpg,Ne,"[283, 36, 294, 43]",19/11/2020,"[370, 48, 417, 57]",7000,"[444, 227, 483, 237]",Si Partculers Gusnity ale per Amount\n\n11) Re...,"[11, 190, 280, 43]"
2,77502028-18e6-4a51-9b7d-0cc5f4a68726.jpg,B202001676,"[620, 499, 720, 510]",07/03/2020,"[180, 891, 262, 912]",15576,"[713, 1967, 751, 1978]",S.No. Parts/OP code _—Part Labor Description H...,"[128, 746, 855, 840]"
3,e8d8c551-7a88-4f7d-bccf-4b205317b21e.jpg,123456789-SA,"[515, 1702, 741, 1729]",14/09/2017,"[1494, 1702, 1564, 1729]",918,"[2200, 3010, 2310, 3041]",Items in this shipment Ordered Shipment Follow...,"[200, 2063, 2310, 3041]"
4,a7e533f9-fea3-453e-b135-6b6d9d3c63a6.jpg,48,"[527, 566, 556, 586]",18/12/2014,"[529, 517, 671, 537]",121,"[1355, 1449, 1454, 1469]",Product Quantity Lda)\nSimple product 1 €100.0...,"[185, 910, 1454, 1469]"
5,4907a532-bf75-4bc4-a08c-06f9d3f00306.png,71912018,"[687, 243, 737, 252]",04/07/2018,"[687, 243, 737, 252]",349.85,"[735, 606, 782, 617]",Description Quantity Rate Options. Total In-fi...,"[45, 397, 782, 685]"
6,19d98817-caf1-4e5a-b8b2-22881ecef5d4.png,2017-40,"[779, 82, 840, 95]",20/07/2017,"[754, 112, 840, 125]",4899.36,"[807, 858, 860, 869]",No. | Name Qty | Rate | Disc. | Amount | % | A...,"[97, 543, 860, 869]"
7,2ec7883e-dafe-4cc3-9836-7314ace98c14.png,2022/000323,"[486, 171, 554, 179]",07/07/2022,"[519, 227, 545, 235]",176.6,"[592, 718, 620, 727]",Description QTY. Price Total Beanie with Logo ...,"[379, 584, 620, 727]"
8,292518c6-bfd4-4096-92a1-7df63faf0860.png,CIV-000676 000007-1,"[693, 415, 737, 424]",30/11/2019,"[696, 162, 730, 173]",321113.82,"[677, 623, 736, 634]",mem DESCRIPTION 'quanriry SALES PRICE DISCOUNT...,"[41, 282, 736, 634]"
9,aacc330f-a0ad-4248-bce9-9eab83818e2b.jpg,,[],04/04/2020,"[478, 466, 514, 467]",,[],,[]


In [33]:
df_test.to_csv('invoice-llm-prediction-final.csv', index=False)

In [34]:
FileLink(r'invoice-llm-prediction-final.csv')

/kaggle/working/invoice-llm-prediction-final.csv